# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\../UCBC 2024/Conf/UCBC2024.conf



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

Loading config file d:\GitHub\ben\src\../UCBC 2024/Conf/UCBC2024.conf


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'K62.Q7542.J832.7'

auction = ["PASS", "2N", "PASS", "3D","PASS", "4C", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 0, dds, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll
DDSolver being loaded version 2.9.0.0 - dds mode 1


[{'call': '4H',
  'insta_score': 0.518,
  'adjustment': 259,
  'expected_score': 429,
  'expected_tricks': 10.8},
 {'call': '4D',
  'insta_score': 0.279,
  'adjustment': 139,
  'expected_score': 333,
  'expected_tricks': 10.7},
 {'call': '4N',
  'insta_score': 0.196,
  'adjustment': 98,
  'expected_score': 269,
  'expected_tricks': 10.8}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['J9xx.Tx.Kx.KJ9xx AQTx.A98x.ATx.AQ 8x.KJ.Q9xx.T8xxx Kxx.Qxxxx.J8xx.x 0.53711 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.8)  \n P-2N-P-3D-P-4C-P-4D-P-4H-P-P-P (11, 11.0)  \n P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (11, 10.8) ',
 'J8x.Jx.Axx.QTxxx Qx.AKTx.KQT9.AKx AT9xx.98.xx.J98x Kxx.Qxxxx.J8xx.x 0.52100 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.5)  \n P-2N-P-3D-P-4C-P-4D-P-4H-P-P-P (11, 10.6)  \n P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (11, 10.5) ',
 'A8x.Jx.9xx.98xxx Jx.AKT8.AKTx.AJx QT9xx.9x.Qx.KQTx Kxx.Qxxxx.J8xx.x 0.51123 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 11.1)  \n P-2N-P-3D-P-4C-P-4D-P-4H-P-P-P (11, 11.2)  \n P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (11, 11.1) ',
 'J8x.x.Q9xx.K8xxx AQ.AJ9x.AKTx.ATx T9xxx.KT8.x.QJ9x Kxx.Qxxxx.J8xx.x 0.50879 \n P-2N-P-3D-P-4C-P-4H-P-P-P (12, 11.9)  \n P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-6H-P-P-P (12, 11.7)  \n P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (12, 11.9) ',
 'AQ9x.8xx.T9.QT9x xx.AKJ9.AKQx.AJx JT8x.T.xxx.K8xxx Kxx.Qxxxx.J8xx.x 0.50879 \n P-2N-P-3D-P-4C-P-4H-P-P-P (11, 10.9) 

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: 0.8261333264802632
Kxx.Qxxxx.J8xx.x QT8xx.AJ9.9x.K9x J.KTxx.AQx.AT8xx A9xx.8.KTxx.QJxx
Kxx.Qxxxx.J8xx.x QT9xxx.x.ATxx.Kx J.AJ9x.Kx.AQT9xx A8x.KT8.Q9x.J8xx
Kxx.Qxxxx.J8xx.x AT98x.9.KQ9x.J8x x.AKTx.ATxx.Kxxx QJxx.J8x.x.AQT9x
Kxx.Qxxxx.J8xx.x AJ98x.K8.Txx.QJx x.AJTx.AKQxx.9xx QTxx.9x.9.AKT8xx
Kxx.Qxxxx.J8xx.x AJT9xx.x.AT.J9xx 8.AKJx.xxx.AKTxx Qxx.T98.KQ9x.Q8x
Kxx.Qxxxx.J8xx.x Jxxxx.AJ.Kx.QJTx A.KTxx.Txxx.AKxx QT98.98.AQ9.98xx
Kxx.Qxxxx.J8xx.x AQJxx.T8.Q.QT9xx 9.AKJx.KT9x.AJ8x T8xx.9x.Axxx.Kxx
Kxx.Qxxxx.J8xx.x AT98x..Axx.JT9xx x.K98x.QT9.AKQ8x QJxx.AJTx.Kxx.xx
Kxx.Qxxxx.J8xx.x AQT8x.8.Txx.Q8xx x.AJ9x.AK9x.AJ9x J9xx.KTx.Qx.KTxx
Kxx.Qxxxx.J8xx.x AQJ9xx.8.T9x.xxx T.K9xx.AQx.AKJT8 8xx.AJT.Kxx.Q9xx
Kxx.Qxxxx.J8xx.x QT8xxx.A9.Txx.9x x.KJ8x.AK.AQT8xx AJ9.Tx.Q9xx.KJxx
Kxx.Qxxxx.J8xx.x QT9xx.JTx.A9x.AJ x.AK9x.Qx.KT8xxx AJ8x.8.KTxx.Q9xx
Kxx.Qxxxx.J8xx.x AQT9x.Ax.Qx.T9xx 8.JT9x.AKTx.AKJ8 Jxxx.K8.9xx.Qxxx
Kxx.Qxxxx.J8xx.x AQT9x.T9x.x.QJ98 .AKJ8.KQ9x.ATxxx J8xxx.x.ATxx.Kxx
Kxx.Qxxxx.J8xx.